<table style="text-align: left; width: 100%;" border="0"
 cellpadding="0" cellspacing="0">

  <tbody>
    <tr align="center" style="height: 1px; background-color: rgb(0, 0, 0);">
      <td><big><big><big><big>Grupo-7</big></big></big></td>
      <td><big><big><big><big><span style="font-family: Calibri; color: white; font-weight: bold;">Laboratorio III</span></big></big></big></big></td>
      <td><img src="https://i.imgur.com/YOQky86.png" title="source: imgur.com" style="width: 250px; height: auto;" /></td>
    </tr>
    <tr align="center">
      <td colspan="3" rowspan="1"
 style="height: 1px; background-color: rgb(68, 68, 100);"></td>
    </tr>
    <tr align="center">
      <td colspan="3" rowspan="1"><big><big><big><big><span
 style="font-family: Calibri;">Data Preparation & Analysis</span></big></big></big></big><br>
      </td></tr>
    <tr align="center">       
      <td colspan="3" rowspan="1" style="height: 1px; background-color: rgb(68, 68, 100);"></td>
    </tr>
  </tbody>
</table>


<span style="font-family: Calibri; font-weight: bold; ">Autores:</span>
<br style="font-family: Calibri; font-style: italic;">
<span style="font-family: Calibri; font-style: italic;">
- Aureliano Chavarria
- Gastón Larregui
- Patricia Nuñez
</span>



<div class="footer">&copy; 2024</div>


jupyter nbconvert --to script xxx.ipynb

# Introducción

Esta notebook fue está diseñada para llevar a cabo diversas tareas esenciales en el proceso de preparación y análisis de datos de ventas. Su contenido abarca una serie de funciones y procedimientos que facilitan la manipulación, evaluación y visualización de los datos. A continuación, se describen brevemente los componentes principales del script:

1. **Conversión y Formateo de Fechas**:
   - Se convierte el formato de las fechas en el DataFrame `df_ventas` al tipo `datetime` de pandas, lo cual es crucial para realizar análisis temporales y manipular los datos de manera eficiente.

2. **Cálculo del Error Total de Pronóstico**:
   - Se incluye una función `calcular_total_forecast_error` que calcula el error total de pronóstico (Total Forecast Error) comparando las ventas reales con las pronosticadas. Esta métrica es fundamental para evaluar la precisión de los modelos predictivos desarrollados.

3. **Análisis y Filtrado de Productos**:
   - Se realiza el filtrado de productos sin categoría (`sinCatego`) y la identificación de productos que faltan en la descripción de productos (`df_productos_descripcion`). Para mantener la integridad de los datos, se crean DataFrames para estos productos faltantes y se completan con valores predeterminados.

4. **Generación de Archivos CSV para PostgreSQL**:
   - Se generan archivos CSV (`sell-in-all-to-postgres.csv` y `tb_productos_descripcion_ac.csv`) que están listos para ser cargados en una base de datos PostgreSQL. Esta etapa es esencial para la preparación de datos y su posterior análisis utilizando herramientas como Grafana.

5. **Visualización de Datos Específicos**:
   - Se incluyen ejemplos para visualizar subconjuntos específicos de datos, permitiendo un análisis más detallado de casos particulares, como productos con un `product_id` específico.

Este script es una herramienta versátil que automatiza y simplifica varias etapas del proceso de análisis de datos, desde la preparación inicial hasta la evaluación y generación de archivos para su posterior explotación en bases de datos y plataformas de visualización.



<hr>

# Prediccion Feb 2020: Lectura Completa y Rapida de ventas
## Dataset sell-z-780-all-LTSM.csv

**Nota:** Este `df_ventas` genera `sell-z-780-all-LTSM.csv` y solo contiene los datos de los 780 productos a predecir.

Luego de ejecutar tenemos el `df_ventas` con: 
- Todas las ventas de los 780 productos a predecir
- Matriz de 780 productos * 36 periodos = 28080 (se completaron con 0 los periodos faltantes para cada producto)
- Columnas con la clasificacion de productos: cat1, cat2, cat3, brand, sku_size, descripcion
- Columna en780 (deberia estar todo en 1 ya que indica que es producto a predecir para febrero 2020)
- Columna cant_periodos: contiene para cada producto la cantidad de periodos que tuvo ventas

In [ ]:
import pandas as pd

# Cargar los datos de ventas y productos a predecir
#sell-z-780-groupBy-Period-ProdId.csv
#ventas_file_path = '../66_Datos/sell-in.txt'
ventas_file_path = '../66_Datos/sell-z-780-groupBy-Period-ProdId.csv'
productos_file_path = '../66_Datos/productos_a_predecir.txt'
productos_descripcion = '../66_Datos/tb_productos_descripcion.txt'


# tb_productos_descripcion.txt es la ultima tabla (∫2024/06/15) que presento con las caracteristicas y categoria de cada producto
df_productos_descripcion = pd.read_csv(productos_descripcion, sep='\t')
df_productos_descripcion = df_productos_descripcion[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'descripcion']]
df_productos_descripcion = df_productos_descripcion.reset_index(drop=True)
# Nota: no encuentro duplicados en productos_descripcion. De todas formas armo la sentencia de eliminarlos ya que pueden aparecer
#       futuras versiones del archivo
# Eliminar los duplicados en la columna 'product_id', manteniendo solo la primera aparición
df_productos_descripcion.drop_duplicates(subset='product_id', keep='first', inplace=True)

#df_ventas_orig = pd.read_csv(ventas_file_path, sep='\t')
df_ventas_orig = pd.read_csv(ventas_file_path)

# Borro columna customer_id 
#   Esta columna viene del archivo csv que agrupo valores por product_id y periodo. No sabemos si customer_id es un valor totalizado.
#   En principio no lo necesito
df_ventas = df_ventas_orig.copy()
del df_ventas['customer_id']

# -------------------------------------------------------
# Agregar Periodos faltantes con valor = 0
#    LSTM necesita toda la serie temporal en la que los productos fueron vendidos para poder predecir a futuro, incluso
#    en meses dde no hubieron ventas. Para estos meses se completa el periodo, product_id y valores a cero para tn (y demas valores tipo numericos)

# Primero convertimos la columna 'periodo' en datetime si no está en ese formato
df_ventas['periodo'] = pd.to_datetime(df_ventas['periodo'], format='%Y%m')

meses = df_ventas.periodo.unique()
prod = df_ventas.product_id.unique()

all_combinations = pd.MultiIndex.from_product([prod, meses], names=['product_id', 'periodo']).to_frame(index=False)

# Unir los datos originales con el DataFrame completo para asegurar que cada producto tenga todos los periodos
df_full = pd.merge(all_combinations, df_ventas, on=['product_id', 'periodo'], how='left')

# Rellenar los valores faltantes con ceros
df_full['tn'] = df_full['tn'].fillna(0)

df_ventas = df_full.copy()


# -------------------------------------------------------
# Agregar la col cant_periodos
#    Cant periodos indica cuantos meses hubo ventas de un product_id
# Filtrar el DataFrame para excluir los períodos donde 'tn' es cero
df_ventas_filtrado = df_ventas[df_ventas['tn'] != 0]

# Agrupar por 'product_id' y contar los períodos únicos en el DataFrame filtrado
cant_periodos = df_ventas_filtrado.groupby('product_id')['periodo'].nunique()

# Agregamos esta información al DataFrame original
df_ventas = df_ventas.merge(cant_periodos.rename('cant_periodos'), on='product_id', how='left')


# -------------------------------------------------------
# Agregamos la descripcion de cada producto (cat1, cat2, cat3, etc)
#   Esto podria ser util en caso de querer incorporar nuevas variables al modelo
df_ventas = pd.merge(df_ventas, df_productos_descripcion, on=['product_id'], how='left')

# Contar la cantidad de NaN en cada columna
cantidad_nans_por_columna = df_ventas.isna().sum()
print('\n--------------------------------------')
print('Cantidad de NaN x Columna:')
print(cantidad_nans_por_columna)

# Rellenar los valores faltantes con ceros
df_ventas = df_ventas.fillna(0)

print('\n--------------------------------------')
print('Completando a 0 los NaN')



# Contar la cantidad de NaN en cada columna
cantidad_nans_por_columna = df_ventas.isna().sum()
print('\n--------------------------------------')
print('Cantidad de NaN x Columna:')
print(cantidad_nans_por_columna)



print('\n--------------------------------------\n')
print('Generando archivo final: ../66_Datos/sell-z-780-all-LTSM.csv')

# En caso de generar el archivo a csv para uso futuro
#df_ventas.to_csv('../66_Datos/sell-z-780-all-LTSM.csv', index=False)
df_ventas.round(1).to_excel('../66_Datos/sell-z-780-all-LTSM.xlsx', index=False)

print('\n--------------------------------------')

## Lecutura rapida de Dataset sell-z-780-all-LTSM.csv

In [ ]:
import pandas as pd

ventas_LTSM_path = '../66_Datos/sell-z-780-all-LTSM.csv'
df_ventas = pd.read_csv(ventas_LTSM_path)

# Convertir la columna 'periodo' a tipo datetime
df_ventas['periodo'] = pd.to_datetime(df_ventas['periodo'], format='%Y-%m-%d')

# Formatear la fecha según el formato deseado
print(df_ventas.info())

df_ventas.head(2)

<hr>

# Formula Total Forecast Error

In [ ]:
import pandas as pd

def calcular_total_forecast_error(actual, forecast):
    """
    Calcula el Total Forecast Error dado un DataFrame con ventas reales y pronosticadas.

    Parámetros:
    actual (pd.Series): Serie con las ventas reales.
    forecast (pd.Series): Serie con las ventas pronosticadas.

    Retorna:
    float: El Total Forecast Error.
    """
    # Calcular el error absoluto
    abs_error = abs(actual - forecast)
    
    # Calcular el Total Forecast Error
    total_forecast_error = abs_error.sum() / actual.sum()
    
    print("\n\n-----------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------")
    print(f'    >>>>>>>>>>>>       Total Forecast Error: {total_forecast_error:.2%}     <<<<<<<<<<<<<<<<<<')
    print("-----------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------\n\n")

    
    return total_forecast_error

# Ejemplo de uso
# Crear un DataFrame de ejemplo
data = {
    'product_id': [1, 2, 3, 4, 5],
    'actual_sales': [100, 150, 200, 250, 300],
    'forecast_sales': [110, 145, 190, 260, 310]
}
df = pd.DataFrame(data)

# Calcular el Total Forecast Error
tfe = calcular_total_forecast_error(df['actual_sales'], df['forecast_sales'])
#print("\n\n-----------------------------------------------------------------------------")
#print("-----------------------------------------------------------------------------")
#print(f'    >>>>>>>>>>>>       Total Forecast Error: {tfe:.2%}     <<<<<<<<<<<<<<<<<<')
#print("-----------------------------------------------------------------------------")
#print("-----------------------------------------------------------------------------\n\n")



$$
\text{Total Forecast Error} = \frac{\sum_{\text{sku}} |\text{Actual Sales} - \text{Forecast Sales}|}{\sum_{\text{sku}} \text{Actual Sales}}
$$



<hr>

# Funcion nombre_file(sfx)
   - Genera el nombre del archivo .csv con prefijo datetime YYYY-MM-DD

In [ ]:
# --------------------------------------
# Funcion nombre_file(sfx)
#   - Genera el nombre del archivo .csv con prefijo datetime YYYY-MM-DD
from datetime import datetime
def nombre_file(sfx):
    # Obtener la fecha y hora actual en el formato requerido
    current_time = datetime.now().strftime('%Y-%m-%d-%H%M-')

    # Path to output dir
    output_dir = '../666_Kaggle/Entregas/'    
    return(output_dir+current_time+sfx+'.csv')
# Indicar el nombre de la prueba
suffix_name = 'MongoAurelio' 

file_to_kaggle = nombre_file(suffix_name)
print(file_to_kaggle)


# ------------------------------------------------
# Esto va al final para escribir el archivo final

# Agrego el tfe2 al suffix del archivo
#   Calculo del total forecast error
#   tfe2 =  calcular_total_forecast_error(all_forecasts['y'], all_forecasts['yhat1'])
#    print(f'Total Forecast Error: {tfe2:.2%}')

tfe2 = 0.123456789
str_tfe2 = "_tfe2_" + str(round(tfe2, 4)) 

# Suffijo general para las dos salidas de archivos
#  Solo cambiar este valor

suffix_general = 'Sufijo-General'  + str_tfe2

# Usar la función nombre_file para asignar el nombre del archivo de salida para kaggle
suffix_to_kaagle_name = suffix_general
file_to_kaggle = nombre_file(suffix_to_kaagle_name)
# Colocar el nombre del df apropiado
#df_aGuardarEnDisco.to_csv(file_to_kaggle, index=False)

#all_forecasts.to_csv(file_to_kaggle+'all', index=False)
print(f'Predicciones ajustadas guardadas en {file_to_kaggle}')

# Fin
# ------------------------------------------------



In [ ]:
# --------------------------------------
# Funcion nombre_file(sfx)
#   - Genera el nombre del archivo .csv con prefijo datetime YYYY-MM-DD
from datetime import datetime
def nombre_file(sfx):
    # Obtener la fecha y hora actual en el formato requerido
    current_time = datetime.now().strftime('%Y-%m-%d-%H%M-')

    # Path to output dir
    output_dir = '../666_Kaggle/Entregas/'    
    return(output_dir+current_time+sfx+'.csv')

# Indicar el nombre de la prueba


# Agrego el tfe2 al suffix del archivo
#   Calculo del total forecast error
#   tfe2 =  calcular_total_forecast_error(all_forecasts['y'], all_forecasts['yhat1'])
#    print(f'Total Forecast Error: {tfe2:.2%}')

tfe2 = 0.123456789
str_tfe2 = "_tfe2_" + str(round(tfe2, 4)) 

suffix_name = 'MongoAurelio' + str_tfe2
file_to_kaggle = nombre_file(suffix_name)
print(file_to_kaggle)
# --------------------------------------


<hr>

#  Sell-in - Datos de Ventas 

### sell-in.txt

El archivo de ventas contiene todos los datos de ventas facilitados por el docente. Es la tabla punto de partida para las ventas. 

Contiene las siguientes columnas:

1. **periodo**: Representa el período de tiempo en formato `yyyymm` (año y mes).
    - Minimum	2017-01-01 00:00:00
    - Maximum	2019-12-01 00:00:00
2. **customer_id**: Identificación única del cliente.
3. **product_id**: Identificación única del producto.
4. **plan_precios_cuidados**: Indicador binario (0 o 1) que señala si el producto está incluido en el plan de precios cuidados.
5. **cust_request_qty**: Cantidad de producto solicitada por el cliente.
6. **cust_request_tn**: Peso en toneladas solicitado por el cliente.
7. **tn**: Peso en toneladas del producto vendido.

Estas columnas proporcionan información detallada sobre las ventas realizadas en distintos períodos, especificando el cliente, el producto, y las cantidades en unidades y en peso.


### sell-z-780-all-prodId.csv
sell-z-780-all-prodId.csv es una vista de la tabla sell-in.txt con solamente los movimientos de los 780 productos que hay que predeicr (productos_a_predecir.txt).
Las ventas de productos por peridos no estan agregadas. Esto es, existen para un mismo producto en un mismo perodo multiples entradas.  
  EJ:

| Periodo | product_id | tn   |
|---------|------------|------|
| 2018-01 | 20001      | 1360 |
| 2018-01 | 20001      | 160  |
| 2018-01 | 20055      | 3360 |

### sell-z-780-groupBy-Period-ProdId.csv
sell-z-780-groupBy-Period-ProdId.csv contiene los mismos datos que sell-z-780-all-prodId.csv pero agrupados por 'periodo' y 'product_id'. Es decir, para un producto que tuvo multiples ventas en un periodo tengo agrupadas las ventas para ese periodo.

Ej. 
| Periodo | product_id | tn   |
|---------|------------|------|
| 2018-01 | 20001      | 1520 |
| 2018-01 | 20055      | 3360 |







In [ ]:
import pandas as pd

# Cargar los datos de ventas y productos a predecir
ventas_file_path = '../66_Datos/sell-in.txt'
productos_file_path = '../66_Datos/productos_a_predecir.txt'
productos_descripcion = '../66_Datos/tb_productos_descripcion.txt'

df_productos_descripcion = pd.read_csv(productos_descripcion, sep='\t')
df_productos_descripcion = df_productos_descripcion[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'descripcion']]
df_productos_descripcion = df_productos_descripcion.reset_index(drop=True)
# Nota: no encuentro duplicados en productos_descripcion. De todas formas armo la sentencia de eliminarlos ya que pueden aparecer
#       futuras versiones del archivo
# Eliminar los duplicados en la columna 'product_id', manteniendo solo la primera aparición
df_productos_descripcion.drop_duplicates(subset='product_id', keep='first', inplace=True)

df_ventas_orig = pd.read_csv(ventas_file_path, sep='\t')

df_ventas = pd.merge(df_ventas_orig, df_productos_descripcion, on=['product_id'], how='left')


df_productos = pd.read_csv(productos_file_path, sep='\t')


# Filtrar las ventas para que solo incluya los productos a predecir
productos_a_predecir = df_productos['product_id'].unique()
sell_780_all = df_ventas[df_ventas['product_id'].isin(productos_a_predecir)]


sell_780_groupBy_Periodo_ProdId = sell_780_all.groupby(['periodo', 'product_id'], as_index=False).sum()

# Opcional: Guardar el DataFrame filtrado en un archivo CSV
#df['ds'] = pd.to_datetime(df['periodo'], format='%Y%m')
df_ventas_toCsv = df_ventas
df_ventas_toCsv['periodo'] = pd.to_datetime(df_ventas_toCsv['periodo'], format='%Y%m')


In [ ]:
# Generar CSVs 
df_ventas_toCsv.to_csv('../66_Datos/sell-in.csv', index=False)
sell_780_all.to_csv('../66_Datos/sell-z-780-all-prodId.csv', index=False)
sell_780_groupBy_Periodo_ProdId.to_csv('../66_Datos/sell-z-780-groupBy-Period-ProdId.csv', index=False)

In [ ]:
df_productos_descripcion = df_productos_descripcion[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'descripcion']]
df_productos_descripcion = df_productos_descripcion.reset_index(drop=True)

# Nota: no encuentro duplicados en productos_descripcion. De todas formas armo la sentencia de eliminarlos ya que pueden aparecer
#       futuras versiones del archivo
# Eliminar los duplicados en la columna 'product_id', manteniendo solo la primera aparición
df_productos_descripcion.drop_duplicates(subset='product_id', keep='first', inplace=True)

In [ ]:
df1 = pd.merge(df_ventas, df_productos_descripcion, on=['product_id'], how='left')


In [ ]:
# CSV de las ventas de todos los 780 con el campo periodo convertido a datetime
#   Este archivo lo uso en Orange y nec que periodo tenga formato fecha
sell_780_all_toDatetime_toCsv = sell_780_all.copy()
sell_780_all_toDatetime_toCsv['periodo'] = pd.to_datetime(sell_780_all_toDatetime_toCsv['periodo'], format='%Y%m')
sell_780_all_toDatetime_toCsv.to_csv('../66_Datos/sell-z-780-all-prodId-toDatetime.csv', index=False)

In [ ]:
sell_780_all.loc[(sell_780_all['product_id'] == 20001) & (sell_780_all['periodo'] == 201701)].head(5)

## sell-z-39-all-prodId + sell-z-39-groupBy-Period-ProdId

Version con 39 productos que representan el 5% para pruebas rapidas



sell-z-39-groupBy-Period-ProdId.csv es una version reducida de los datos de ventas (con el 5% de productos a predecir= y contiene los datos de ventas agrupados por 'periodo' y 'product_id'. Es decir, para un producto que tuvo multiples ventas en un periodo tengo agrupadas las ventas para ese periodo.

Ej. 
| Periodo | product_id | tn   |
|---------|------------|------|
| 201801 | 20001      | 1520 |
| 201801 | 20055      | 3360 |
| 201802 | 20011      | 1320 |
| 201802 | 20021      | 1220 |
| 201802 | 20031      | 120 |



In [ ]:
import pandas as pd

# Cargar los datos de ventas y productos a predecir
ventas_file_path = '../66_Datos/sell-in.txt'
productos_file_path = '../66_Datos/productos_a_predecir-39.txt'

df_ventas = pd.read_csv(ventas_file_path, sep='\t')
df_productos = pd.read_csv(productos_file_path, sep='\t')

# Filtrar las ventas para que solo incluya los productos a predecir
productos_a_predecir = df_productos['product_id'].unique()
sell_39_all = df_ventas[df_ventas['product_id'].isin(productos_a_predecir)]


sell_39_groupBy_Periodo_ProdId = sell_39_all.groupby(['periodo', 'product_id'], as_index=False).sum()

# Opcional: Guardar el DataFrame filtrado en un archivo CSV
sell_39_all.to_csv('../66_Datos/sell-z-39-all-prodId.csv', index=False)
sell_39_groupBy_Periodo_ProdId.to_csv('../66_Datos/sell-z-39-groupBy-Period-ProdId.csv', index=False)

# EDA con Ydata-Profiling 

Este código carga un conjunto de datos de ventas desde un archivo CSV y utiliza la herramienta `ydata_profiling` para generar un informe detallado de perfilado de datos, que se guarda en un archivo HTML para su posterior análisis y visualización.

In [ ]:
from ydata_profiling import ProfileReport

ventas_LTSM_path = './66_Datos/sell-z-780-all-LTSM.csv'
df_ventas = pd.read_csv(ventas_LTSM_path)

profile = ProfileReport(df_ventas, tsmode=False, title="Profiling Report")

profile.to_file("./EDA_ydataProfiling/eda-yprof.html")

profile = ProfileReport(df_ventas, tsmode=True, title="Profiling Report")

profile.to_file("./EDA_ydataProfiling/eda-yprof-ts.html")

# Misc Temp


In [ ]:
print(range(2021, 2030))

In [ ]:
import pandas as pd
import numpy as np

# Crear un DataFrame con fechas desde 2021 hasta 2039
f2 = pd.DataFrame({
    'ds': pd.to_datetime([str(x) for x in range(2021, 2040)], format='%Y'),
    'y': np.nan
})

print(f2)

In [ ]:
f2

In [ ]:
import pandas as pd
import numpy as np

# 2021 a 2090
futuro = pd.DataFrame({
    'ds': pd.to_datetime([x for x in range(2021, 2025)], format='%Y'),
    'y': np.nan
})

In [ ]:
futuro

In [ ]:
pd.to_datetime([x for x in range(2021, 2025)], format='%Y')

# Ventas To Postgres

Generar archivo sell-in-to-Postgres con:
 1. Todas las ventas registradas en sell-in.txt
 2. Agregar columna `en780` indicando con 1 aquellas filas que contienen product_id perteneciente al listado de 780 productos a predecir (productos_a_predecir.csv)
 3. Merge de las ventas en sell-in.txt con la descripcion de productos en tb_productos_descripcion.txt (join by product_id)
 4. Agregar columna `cant_periodos`, y en cada celda agregar la cantidad de meses que tuvo ventas el product_id de la fila. Por ejemplo si el product_id 20001 tuvo ventas durante los 36 meses, en la columna cant_periodos debe ir 36. 
 5. Exportar a csv  sell-in-all-to-postgres.csv


Como identificar productos vendidos durante los 36 meses

In [ ]:
import pandas as pd

# Cargar los datos de ventas y productos a predecir
ventas_file_path = '../66_Datos/sell-in.txt'
productos_file_path = '../66_Datos/productos_a_predecir.txt'
productos_descripcion = '../66_Datos/tb_productos_descripcion.txt'


# tb_productos_descripcion.txt es la ultima tabla (2024/06/15) que presento con las caracteristicas y categoria de cada producto
df_productos_descripcion = pd.read_csv(productos_descripcion, sep='\t')
df_productos_descripcion = df_productos_descripcion[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'descripcion']]
df_productos_descripcion = df_productos_descripcion.reset_index(drop=True)
# Nota: no encuentro duplicados en productos_descripcion. De todas formas armo la sentencia de eliminarlos ya que pueden aparecer
#       futuras versiones del archivo
# Eliminar los duplicados en la columna 'product_id', manteniendo solo la primera aparición
df_productos_descripcion.drop_duplicates(subset='product_id', keep='first', inplace=True)

df_ventas_orig = pd.read_csv(ventas_file_path, sep='\t')

df_ventas = pd.merge(df_ventas_orig, df_productos_descripcion, on=['product_id'], how='left')

# Cargamos los productos a predecir (780)
df_productos = pd.read_csv(productos_file_path, sep='\t')


# Filtrar las ventas para que solo incluya los productos a predecir. Va .unique() 
productos_a_predecir = df_productos['product_id'].unique()

# Agregar la columna "en780" a df_ventas
df_ventas['en780'] = df_ventas['product_id'].apply(lambda x: 1 if x in productos_a_predecir else 0)

# Agregar columna `cant_periodos`
# Primero convertimos la columna 'periodo' en datetime si no está en ese formato
df_ventas['periodo'] = pd.to_datetime(df_ventas['periodo'], format='%Y%m')

# Agrupamos por 'product_id' y contamos los periodos únicos
cant_periodos = df_ventas.groupby('product_id')['periodo'].nunique()

# Agregamos esta información al DataFrame original
df_ventas = df_ventas.merge(cant_periodos.rename('cant_periodos'), on='product_id', how='left')


## Valores nulos df_productos_descripcion y df_ventas

Resulta que en la tabla productos_descripcion.txt hay un listado de 45 productos vendidos durante los 36 peridos que no figuran.
 - Voy a completar esos nulos tanto en la tabla que llevo a postgres sell_all (sell-in-all-to-postgres.csv) como productos2 (tb_productos_descripcion_ac.csv)
 - Estoy completando las variables cat1, cat2, cat3, brand y descripción con el string 'sinCatego'
 - Estoy completando la variable sku_size con 0. 

In [ ]:
# Verificar si hay valores nulos en cat1, cat2, cat3, brand, sku_size, descripcion
nulos_cat1 = df_ventas['cat1'].isnull().sum()
nulos_cat2 = df_ventas['cat2'].isnull().sum()
nulos_cat3 = df_ventas['cat3'].isnull().sum()
nulos_brand = df_ventas['brand'].isnull().sum()
nulos_sku_size = df_ventas['sku_size'].isnull().sum()
nulos_descripcion = df_ventas['descripcion'].isnull().sum()

print(f'Valores nulos en cat1: {nulos_cat1}')
print(f'Valores nulos en cat2: {nulos_cat2}')
print(f'Valores nulos en cat3: {nulos_cat3}')
print(f'Valores nulos en brand: {nulos_brand}')
print(f'Valores nulos en sku_size: {nulos_sku_size}')
print(f'Valores nulos en descripcion: {nulos_descripcion}')


# Reemplazar valores nulos en columnas tipo object con "sinCatego"
cols_object = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion']
for col in cols_object:
    df_ventas[col].fillna('sinCatego', inplace=True)

# Reemplazar valores nulos en columnas numéricas con 0
cols_numeric = ['sku_size']
for col in cols_numeric:
    df_ventas[col].fillna(0, inplace=True)


# Verificar si hay valores nulos en cat1, cat2, cat3, brand, sku_size, descripcion
nulos_cat1 = df_ventas['cat1'].isnull().sum()
nulos_cat2 = df_ventas['cat2'].isnull().sum()
nulos_cat3 = df_ventas['cat3'].isnull().sum()
nulos_brand = df_ventas['brand'].isnull().sum()
nulos_sku_size = df_ventas['sku_size'].isnull().sum()
nulos_descripcion = df_ventas['descripcion'].isnull().sum()

print(f'Valores nulos en cat1: {nulos_cat1}')
print(f'Valores nulos en cat2: {nulos_cat2}')
print(f'Valores nulos en cat3: {nulos_cat3}')
print(f'Valores nulos en brand: {nulos_brand}')
print(f'Valores nulos en sku_size: {nulos_sku_size}')
print(f'Valores nulos en descripcion: {nulos_descripcion}')


print('--------------------------------------')
print('--------------------------------------')



# Filtrar los productos en df_ventas donde cat1 es 'sinCatego'
productos_sin_categoria = df_ventas[df_ventas['cat1'] == 'sinCatego']['product_id'].unique()

# Encontrar los product_id que faltan en df_productos_descripcion
productos_descripcion_ids = df_productos_descripcion['product_id'].unique()
productos_faltantes = set(productos_sin_categoria) - set(productos_descripcion_ids)

# Mostrar los product_id que faltan en df_productos_descripcion
print('Product IDs que faltan en df_productos_descripcion:')
print(productos_faltantes)


print('--------------------------------------')
print('--------------------------------------')


# Crear un DataFrame con los productos faltantes
productos_faltantes_df = pd.DataFrame({
    'product_id': list(productos_faltantes),
    'cat1': 'sinCatego',
    'cat2': 'sinCatego',
    'cat3': 'sinCatego',
    'brand': 'sinCatego',
    'sku_size': 0,
    'descripcion': 'sinCatego'
})

# Concatenar el DataFrame de productos faltantes con df_productos_descripcion
df_productos_descripcion_completo = pd.concat([df_productos_descripcion, productos_faltantes_df], ignore_index=True)

# Verificar el resultado
print(df_productos_descripcion_completo)


In [ ]:
# Opcional: Guardar el DataFrame filtrado en un archivo CSV
#df['ds'] = pd.to_datetime(df['periodo'], format='%Y%m')
df_ventas_toCsv = df_ventas
#df_ventas_toCsv['periodo'] = pd.to_datetime(df_ventas_toCsv['periodo'], format='%Y%m')

# Genero el csv para crear en pg z_l3.sell_all
df_ventas_toCsv.to_csv('../66_Datos/sell-in-all-to-postgres.csv', index=False)


# Genero el csv para crear en pg z_l3.productos2
df_productos_descripcion_completo.to_csv('../66_Datos/tb_productos_descripcion_ac.csv', index=False)


In [ ]:
df_ventas.loc[df_ventas.product_id == 20808].head()